1. **라이브러리 임포트**:
   - `os`: 파일 입출력을 위해 사용.
   - `sqlite3`: SQLite 데이터베이스를 관리하기 위해 사용.
   - `openai`: OpenAI API를 사용하여 텍스트 번역을 처리.
   - `pandas`: 데이터를 DataFrame으로 불러와 처리 및 분석하기 위해 사용.

2. **OpenAI API 설정**:
   - API 키는 환경 변수에서 불러와 설정.
   - OpenAI API 클라이언트를 통해 텍스트 번역 작업을 처리.

3. **텍스트 파일 읽기 및 분리**:
   - 텍스트 파일을 개행 문자 기준으로 분리한 후, 텍스트를 일정 길이로 청크(1000자 단위)로 분할.

4. **데이터베이스 생성 및 테이블 설정**:
   - SQLite 데이터베이스를 생성하고, 텍스트 청크, 텍스트 길이, 번역 결과 등을 저장할 테이블(`chunks`)을 생성.

5. **데이터베이스에 데이터 삽입**:
   - 분할된 텍스트 청크를 데이터베이스에 삽입. 이때 번역 전 텍스트(`original_text`), 텍스트 길이(`length`), 번역 후 텍스트(`translation`), 번역된 텍스트의 길이(`translation_length`)를 저장.

6. **OpenAI API를 활용한 텍스트 번역**:
   - 번역 함수(`translate_text`)는 OpenAI의 GPT-4o-mini 모델을 사용하여 텍스트를 영어에서 한국어로 번역.
   - 번역되지 않은 텍스트 청크를 데이터베이스에서 가져와, 번역 후 그 결과를 업데이트.

7. **데이터 프레임 확인**:
   - SQLite 데이터베이스에서 `pandas`를 사용하여 데이터를 읽어오고, 번역된 결과를 확인. 이를 통해 번역 전후 텍스트 길이 비교 및 데이터 통계를 출력.

8. **번역 결과 저장**:
   - 번역된 텍스트를 하나의 파일(`translation_results.txt`)로 저장.
   
### Prompt
* 텍스트 파일을 열어 개행 기준으로 데이터를 나누고 sqlite 파일로 저장할것
* 이 때 컬럼을  text, length, gpt4omini_trans, gpt4omini_trans 로 만들고 news_text 에는 나눈 텍스트 데이터, length 에는 해당 텍스트 길이가 들어가도록 하고 나머지 컬럼은 비워둘 것
* 주석은 한국어로 작성하고 데이터프레임을 불러와서 df 변수에 담고 확인하는 코드를 함께 작성할 것

* 모델 목록 : https://platform.openai.com/docs/models
* 플레이그라운드에서 미리 사용해 보기 : https://platform.openai.com/playground/chat?models=gpt-4o
* API 키 발급 : https://platform.openai.com/api-keys
* 과금 확인 : https://platform.openai.com/usage
* 공식API : [openai-cookbook/examples/book\_translation/translate\_latex\_book.ipynb at main · openai/openai-cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/book_translation/translate_latex_book.ipynb)

https://www.youtube.com/watch?v=yp6WuHFhYCo

In [ ]:
# !pip install -Uq openai

In [ ]:
import os
import sqlite3
from openai import OpenAI

# OpenAI API 키 설정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_first_txt_file(directory='.'):
    txt_files = [f for f in os.listdir(directory) if f.endswith('yp6WuHFhYCo_subtitle.txt')]
    txt_files.sort()  # 파일명 알파벳 순으로 정렬
    return txt_files[0] if txt_files else None

def create_chunks(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def create_db_and_table():
    conn = sqlite3.connect('text_chunks.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS chunks
                      (id INTEGER PRIMARY KEY,
                       original_text TEXT,
                       length INTEGER,
                       translation TEXT,
                       translation_length INTEGER)''')
    conn.commit()
    return conn, cursor

def insert_chunks(cursor, chunks):
    for chunk in chunks:
        cursor.execute('''INSERT INTO chunks (original_text, length, translation)
                          VALUES (?, ?, ?)''', (chunk, len(chunk), None))

def translate_text(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a translator. Translate the given text from English to Korean."},
            {"role": "user", "content": f"Translate the following text to Korean: {text}"}
        ]
    )
    return response.choices[0].message.content.strip()

def update_translations(conn, cursor):
    cursor.execute("SELECT id, original_text FROM chunks WHERE translation IS NULL")
    rows = cursor.fetchall()
    
    for row in rows:
        id, original_text = row
        translated = translate_text(original_text)
        translation_length = len(translated)
        
        cursor.execute('''UPDATE chunks 
                          SET translation = ?, translation_length = ? 
                          WHERE id = ?''', (translated, translation_length, id))
        conn.commit()  # 각 번역 후 커밋
        print(f"Translated chunk {id}")

def main():
    # 첫 번째 .txt 파일 찾기
    first_txt_file = get_first_txt_file()
    if not first_txt_file:
        print("No .txt files found in the current directory.")
        return

    # 텍스트 파일 읽기
    with open(first_txt_file, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # 줄바꿈 문자로 분리
    lines = text.split('\n')
    
    # 1000자 단위로 청크 생성
    chunks = []
    current_chunk = ""
    for line in lines:
        if len(current_chunk) + len(line) + 1 > 1000:
            chunks.extend(create_chunks(current_chunk))
            current_chunk = line + '\n'
        else:
            current_chunk += line + '\n'
    
    if current_chunk:
        chunks.extend(create_chunks(current_chunk))
    
    # DB 생성 및 연결
    conn, cursor = create_db_and_table()
    
    # 청크 삽입
    insert_chunks(cursor, chunks)
    
    print(f"파일 '{first_txt_file}'에서 총 {len(chunks)}개의 청크가 DB에 저장되었습니다.")

    # 번역 수행
    print("번역을 시작합니다...")
    update_translations(conn, cursor)
    print("번역이 완료되었습니다.")

    # 결과 출력 (예시로 처음 5개 행)
    cursor.execute("SELECT * FROM chunks LIMIT 5")
    for row in cursor.fetchall():
        print(row)

    # 연결 종료
    conn.close()

if __name__ == "__main__":
    main()

## 번역결과 판다스 데이터프레임으로 확인하기

In [ ]:
# 저장된 데이터를 pandas DataFrame으로 읽어오기
import pandas as pd

# SQLite 데이터베이스에 다시 연결
conn = sqlite3.connect('text_chunks.db')

# SQL 쿼리를 사용하여 'chunks' 테이블의 모든 데이터를 DataFrame으로 읽어오기
df = pd.read_sql_query("SELECT * FROM chunks", conn)

# 연결 종료
conn.close()

# DataFrame의 처음 몇 행 출력하여 확인
df.head()

In [ ]:
# DataFrame의 정보 출력
df.info()

## 번역 전 후 길이 비교

In [ ]:
(df["length"] - df["translation_length"]).to_frame().hist()

In [ ]:
df.describe()

## 하나의 파일로 만들기

In [ ]:
translation = "\n\n".join(df["translation"].tolist())
translation[:100]

In [ ]:
file_path = "translation_results.txt"

# 텍스트 파일로 저장
with open(file_path, "w", encoding="utf-8") as file:
    file.write(translation)